In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.llms import OllamaLLM
from langchain_ollama.chat_models import ChatOllama


from pydantic import BaseModel, Field

class DataSchema(BaseModel):
    """Estrutura dados de correção de redação"""
    nota: float  = Field(description="Nota da redação", examples=[100.0, 150.0, 200.0])
    feedback: str = Field(description="Feedback da redação", examples=["Muito bom!", "Pode melhorar!"])
    justificativa: str = Field(description="Justificativa da nota", examples=["A redação atende a todos os critérios.", "Faltou clareza em alguns pontos."])

In [5]:
model_name_gemma3n_4b = "gemma3n:e4b"
model_name_gemma3n_2b = "gemma3n:e2b"

llm = ChatOllama(model=model_name_gemma3n_4b, 
                 temperature=0.0)
                #  num_gpu=1)

llm_with_tool = llm.bind_tools([DataSchema])#, strict=True)

system_prompt = """
Você é um estruturador de saídas de correção de redação.
Sua tarefa é analisar o texto da correção da redação e gerar uma saída estruturada com os seguintes campos:
- nota: A nota da redação, que deve ser um número entre 0 e 200.
- feedback: Um feedback breve sobre a redação, que pode ser positivo ou negativo.
- justificativa: Uma justificativa para a nota atribuída, explicando os critérios de avaliação utilizados.
"""

prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt), 
            ("human", "Texto de saida: \n\n {text}")
        ]
)

chain = prompt | llm_with_tool #| StrOutputParser()

In [6]:
query = """

## Avaliação da Competência 1 (0 a 200)

**Nota: 165**

**Justificativa:**

A redação demonstra um bom domínio da norma padrão da língua portuguesa, com a maioria das construções gramaticais corretas. No entanto, existem alguns pontos que merecem atenção para elevar a nota.

**Pontos Fortes:**

*   **Vocabulário:** O texto apresenta um vocabulário rico e adequado ao tema, utilizando termos como "invisibilidade", "territórios", "políticas públicas", "estereótipos" e "inclusiva".
*   **Coesão e Coerência:** A argumentação é consistente e as ideias se conectam de forma lógica, utilizando conectivos adequados (ex: "além disso", "portanto").
*   **Organização:** A estrutura do texto é clara, com introdução, desenvolvimento e conclusão bem definidos.
*   **Conhecimento do Tema:** O texto demonstra compreensão do tema, abordando os desafios da valorização das comunidades tradicionais no Brasil e as causas dessa invisibilidade.

**Pontos a Melhorar:**

*   **Concordância Verbal e Nominal:** Em alguns trechos, a concordância verbal e nominal poderia ser mais precisa. Por exemplo, na frase "A Constituição de 1988 recoñece os direitos territoriais...", a forma verbal "reconhece" está correta, mas a expressão "reconhece" poderia ser substituída por uma forma mais elegante, como "garante" ou "assegura".
*   **Regência Verbal:** Há alguns casos de regência verbal incorreta. Por exemplo, "A ausência de políticas públicas eficazes também dificulta o acesso dessas populações à saúde..." deveria ser "A ausência de políticas públicas eficazes também dificulta o acesso dessas populações à saúde, *que*...".
*   **Colocação Pronominal:** A colocação pronominal em algumas frases poderia ser mais natural. Por exemplo, "Apesar de sua importância histórica, cultu**r**al e ambiental, essas populações ainda enfrenta**m** uma série de desafios..." poderia ser "Apesar de sua importância histórica, cultural e ambiental, essas populações ainda enfrentam uma série de desafios...".
*   **Impessoalidade:** Em alguns momentos, a redação poderia se beneficiar da utilização de verbos impessoais para evitar repetições e tornar o texto mais formal.
*   **Voz Passiva:** A utilização excessiva da voz passiva pode tornar o texto menos direto e mais complexo.

**Feedback Construtivo:**

Para melhorar sua escrita, concentre-se em revisar cuidadosamente a concordância verbal e nominal, a regência verbal e a colocação pronominal. Preste atenção à construção das frases e tente evitar a voz passiva sempre que possível. Além disso, pratique a leitura de textos bem escritos para identificar as estruturas gramaticais mais utilizadas e aprender novas palavras e expressões.

**Sugestão de Melhoria:**

Na conclusão, tente ser mais conciso e direto ao apresentar as soluções propostas. Em vez de repetir a ideia de que um esforço coletivo é necessário, você poderia apresentar exemplos concretos de ações que podem ser tomadas para combater a invisibilidade das comunidades tradicionais.

Espero que esta avaliação e feedback sejam úteis para você!

"""

# response = chain.invoke({"input": query})

# async for response in chain.astream({"text": query}):
#     print(response, end="", flush=True)

response = chain.invoke({"text": query})

ResponseError: registry.ollama.ai/library/gemma3n:e4b does not support tools (status code: 400)